In [ ]:
# import os, cv2, faiss, torch, numpy as np
# from PIL import Image
# import open_clip
# from tqdm import tqdm
# from IPython.display import display

# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# model, _, preprocess = open_clip.create_model_and_transforms(
#     "ViT-g-14", pretrained="laion2b_s34b_b88k", device=DEVICE
# )
# tokenizer = open_clip.get_tokenizer("ViT-g-14")  # <-- thêm dòng này
# model.eval()

# def sample_frames(video_path, every_ms=500, max_frames=None,
#                   time_window="first_half", start_sec=None, end_sec=None):
#     """
#     time_window: "first_half" | "second_half" | None
#     Hoặc chỉ định start_sec/end_sec (ưu tiên hơn time_window).
#     """
#     cap = cv2.VideoCapture(video_path)
#     assert cap.isOpened(), f"Không mở được video: {video_path}"
#     fps = cap.get(cv2.CAP_PROP_FPS) or 25
#     n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
#     dur = n_frames / fps if n_frames > 0 else None

#     # Tính khoảng đọc
#     if start_sec is None and end_sec is None and dur is not None:
#         if time_window == "first_half":
#             start_sec, end_sec = 0.0, dur * 0.5
#         elif time_window == "second_half":
#             start_sec, end_sec = dur * 0.5, dur
#         else:
#             start_sec, end_sec = 0.0, dur  # toàn bộ
#     elif start_sec is None and end_sec is None:
#         # không biết duration -> đọc toàn bộ (hoặc dùng max_frames)
#         start_sec, end_sec = 0.0, float("inf")

#     # Seek nhanh tới điểm bắt đầu (nếu biết fps & frame_count)
#     if dur is not None:
#         start_frame = int(start_sec * fps)
#         cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

#     step = max(1, int(fps * (every_ms/1000.0)))
#     frames, times = [], []
#     i = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

#     with tqdm(desc="Đọc video (có giới hạn)", unit="f") as pbar:
#         while True:
#             # Dừng nếu đã vượt end_sec
#             cur_sec = cap.get(cv2.CAP_PROP_POS_MSEC)/1000.0
#             if cur_sec and cur_sec > end_sec: break

#             ret, fr = cap.read()
#             if not ret: break
#             if i % step == 0:
#                 rgb = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
#                 frames.append(rgb)
#                 times.append(cap.get(cv2.CAP_PROP_POS_MSEC)/1000.0)
#                 if max_frames and len(frames) >= max_frames:
#                     break
#             i += 1
#             pbar.update(1)

#     cap.release()
#     return frames, times

# @torch.no_grad()
# def embed_images(frames, batch_size=64):
#     vecs = []
#     for i in range(0, len(frames), batch_size):
#         batch = [preprocess(Image.fromarray(x)) for x in frames[i:i+batch_size]]
#         batch = torch.stack(batch).to(DEVICE)
#         feats = model.encode_image(batch)
#         feats = feats / feats.norm(dim=-1, keepdim=True)
#         vecs.append(feats.float().cpu().numpy())
#     return np.vstack(vecs)

# @torch.no_grad()
# def embed_query_vi(text, en_hint=None):
#     texts = [text] + ([en_hint] if en_hint else [])
#     toks = tokenizer(texts).to(DEVICE)
#     feats = model.encode_text(toks)
#     feats = feats / feats.norm(dim=-1, keepdim=True)
#     feat = torch.max(feats, dim=0).values  # lấy biến thể tốt nhất
#     return feat.float().cpu().numpy()[None, :]

# def build_index(vecs):
#     faiss.normalize_L2(vecs)
#     idx = faiss.IndexFlatIP(vecs.shape[1])
#     idx.add(vecs)
#     return idx

# def search_frames(video_path, query_vi, query_en_hint=None,
#                   every_ms=500, topk=10, save_dir="hits",
#                   time_window="first_half", start_sec=None, end_sec=None):
#     os.makedirs(save_dir, exist_ok=True)
#     frames, times = sample_frames(
#         video_path, every_ms=every_ms, time_window=time_window,
#         start_sec=start_sec, end_sec=end_sec
#     )
#     print(f"Số frame sample: {len(frames)}")
#     if not frames: return []

#     img_vecs = embed_images(frames)
#     index = build_index(img_vecs)
#     qv = embed_query_vi(query_vi, query_en_hint)

#     D, I = index.search(qv, topk)
#     I, D = I[0].tolist(), D[0].tolist()

#     base = os.path.splitext(os.path.basename(video_path))[0]
#     results = []
#     for rank, (idx, score) in enumerate(zip(I, D), 1):
#         t = times[idx]
#         thumb = Image.fromarray(frames[idx])
#         out = os.path.join(save_dir, f"{base}_rank{rank:02d}_t{t:.2f}s_{score:.3f}.jpg")
#         thumb.save(out, quality=92)
#         results.append({
#             "rank": rank, "time_sec": float(t), "similarity": float(score),
#             "thumb": out, "frame_index": int(idx)
#         })
#     return results



In [ ]:
# -*- coding: utf-8 -*-
import os, cv2, faiss, torch, numpy as np
from PIL import Image
from tqdm import tqdm
from IPython.display import display

# ======= CHỌN BACKEND =======
# "openclip" (như bạn đang dùng) hoặc "siglip2"
BACKEND = os.environ.get("MM_BACKEND", "siglip2")  # "openclip" | "siglip2"

# ======= LOAD MODEL =======
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

if BACKEND == "openclip":
    import open_clip
    OC_MODEL = "ViT-g-14"
    OC_PRETRAINED = "laion2b_s34b_b88k"
    model, _, preprocess = open_clip.create_model_and_transforms(
        OC_MODEL, pretrained=OC_PRETRAINED, device=DEVICE
    )
    tokenizer = open_clip.get_tokenizer(OC_MODEL)
    model.eval()

elif BACKEND == "siglip2":
    # pip install -U transformers accelerate bitsandbytes
    from transformers import AutoProcessor, AutoModel
    CKPT = os.environ.get("SIGLIP2_CKPT", "google/siglip2-giant-opt-patch16-384")
    processor = AutoProcessor.from_pretrained(CKPT)
    model = AutoModel.from_pretrained(CKPT, device_map="auto").eval()
else:
    raise ValueError("BACKEND phải là 'openclip' hoặc 'siglip2'.")

# ======= VIDEO FRAME SAMPLER =======
def sample_frames(video_path, every_ms=500, max_frames=None,
                  time_window="first_half", start_sec=None, end_sec=None):
    """
    time_window: "first_half" | "second_half" | None
    Hoặc chỉ định start_sec/end_sec (ưu tiên hơn time_window).
    """
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), f"Không mở được video: {video_path}"
    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    dur = (n_frames / fps) if n_frames > 0 else None

    # Tính khoảng đọc
    if start_sec is None and end_sec is None and dur is not None:
        if time_window == "first_half":
            start_sec, end_sec = 0.0, dur * 0.5
        elif time_window == "second_half":
            start_sec, end_sec = dur * 0.5, dur
        else:
            start_sec, end_sec = 0.0, dur  # toàn bộ
    elif start_sec is None and end_sec is None:
        start_sec, end_sec = 0.0, float("inf")

    # Seek tới frame bắt đầu (nếu biết)
    if dur is not None:
        start_frame = int(start_sec * fps)
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    step = max(1, int(fps * (every_ms / 1000.0)))
    frames, times = [], []
    i = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    with tqdm(desc="Đọc video (giới hạn)", unit="f") as pbar:
        while True:
            # Kiểm tra dừng theo end_sec (nếu POS_MSEC khả dụng)
            pos_msec = cap.get(cv2.CAP_PROP_POS_MSEC)
            if pos_msec > 0:
                cur_sec = pos_msec / 1000.0
                if cur_sec > end_sec:
                    break

            ret, fr = cap.read()
            if not ret:
                break

            if i % step == 0:
                rgb = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
                frames.append(rgb)

                # Lấy thời gian hiện tại sau khi read; fallback nếu =0
                pos_msec2 = cap.get(cv2.CAP_PROP_POS_MSEC)
                if pos_msec2 and pos_msec2 > 0:
                    t_sec = pos_msec2 / 1000.0
                else:
                    # Fallback: ước lượng theo frame index
                    t_sec = i / fps
                times.append(t_sec)

                if max_frames and len(frames) >= max_frames:
                    break

            i += 1
            pbar.update(1)

    cap.release()
    return frames, times

# ======= EMBEDDING FUNCS (tuỳ BACKEND) =======
@torch.no_grad()
def embed_images(frames, batch_size=64):
    """
    frames: list[np.ndarray(H,W,3) in RGB]
    return: np.ndarray [N, D] đã L2-norm
    """
    if BACKEND == "openclip":
        vecs = []
        for i in range(0, len(frames), batch_size):
            batch = [preprocess(Image.fromarray(x)) for x in frames[i:i+batch_size]]
            batch = torch.stack(batch).to(DEVICE)
            feats = model.encode_image(batch)
            feats = feats / feats.norm(dim=-1, keepdim=True)
            vecs.append(feats.float().cpu().numpy())
        return np.vstack(vecs)

    elif BACKEND == "siglip2":
        from PIL import Image as _Image
        vecs = []
        pil_frames = [_Image.fromarray(x) for x in frames]
        for i in range(0, len(pil_frames), batch_size):
            batch = pil_frames[i:i+batch_size]
            inputs = processor(images=batch, return_tensors="pt").to(model.device)
            img_feats = model.get_image_features(**inputs)  # (B, D)
            img_feats = img_feats / img_feats.norm(dim=-1, keepdim=True)
            vecs.append(img_feats.float().cpu().numpy())
        return np.vstack(vecs)

@torch.no_grad()
def embed_query_vi(text, en_hint=None):
    """
    Trả về vector 1xD đã L2-norm; nếu có en_hint -> lấy max giữa 2 biến thể.
    """
    if BACKEND == "openclip":
        texts = [text] + ([en_hint] if en_hint else [])
        toks = tokenizer(texts).to(DEVICE)
        feats = model.encode_text(toks)
        feats = feats / feats.norm(dim=-1, keepdim=True)
        feat = torch.max(feats, dim=0).values
        return feat.float().cpu().numpy()[None, :]

    elif BACKEND == "siglip2":
        texts = [text.lower()] + ([en_hint.lower()] if en_hint else [])
        inputs = processor(
            text=texts, return_tensors="pt",
            padding="max_length", max_length=64
        ).to(model.device)
        txt_feats = model.get_text_features(**inputs)
        txt_feats = txt_feats / txt_feats.norm(dim=-1, keepdim=True)
        feat = torch.max(txt_feats, dim=0).values
        return feat.float().cpu().numpy()[None, :]

# ======= FAISS =======
def build_index(vecs: np.ndarray):
    faiss.normalize_L2(vecs)  # an toàn thêm
    idx = faiss.IndexFlatIP(vecs.shape[1])
    idx.add(vecs)
    return idx

# ======= SEARCH =======
def search_frames(video_path, query_vi, query_en_hint=None,
                  every_ms=500, topk=10, save_dir="hits",
                  time_window="first_half", start_sec=None, end_sec=None,
                  show=True):
    os.makedirs(save_dir, exist_ok=True)
    frames, times = sample_frames(
        video_path, every_ms=every_ms, time_window=time_window,
        start_sec=start_sec, end_sec=end_sec
    )
    print(f"Số frame sample: {len(frames)}")
    if not frames:
        return []

    print("Nhúng ảnh...")
    img_vecs = embed_images(frames)
    index = build_index(img_vecs)

    print("Nhúng truy vấn...")
    qv = embed_query_vi(query_vi, query_en_hint)

    print("Tìm top-k...")
    D, I = index.search(qv.astype("float32"), topk)
    I, D = I[0].tolist(), D[0].tolist()

    base = os.path.splitext(os.path.basename(video_path))[0]
    results = []
    for rank, (idx, score) in enumerate(zip(I, D), 1):
        t = times[idx]
        thumb = Image.fromarray(frames[idx])
        out = os.path.join(save_dir, f"{base}_rank{rank:02d}_t{t:.2f}s_{score:.3f}.jpg")
        thumb.save(out, quality=92)
        results.append({
            "rank": rank, "time_sec": float(t), "similarity": float(score),
            "thumb": out, "frame_index": int(idx)
        })
        if show:
            print(f"Rank {rank} | {t:.2f}s | score {score:.3f} | {out}")
            display(thumb)

    return results

# # ======= DEMO (tuỳ chọn) =======
# if __name__ == "__main__":
#     video_path = "sample.mp4"
#     query_vi = "người đội mũ bảo hiểm đi xe máy trên đường"
#     query_en = "a person wearing a helmet riding a motorbike on a street"

#     hits = search_frames(
#         video_path, query_vi, query_en_hint=query_en,
#         every_ms=500, topk=10, time_window="first_half", show=False
#     )
#     print("\nKết quả:")
#     for h in hits: print(h)


In [ ]:
if __name__ == "__main__":
    video_path = "/kaggle/input/video-small/L21_V001.mp4"  # <-- đổi thành video của bạn
    query_vi = "Nhiều người mặc áo xanh dương trong phòng thí nghiệm"
    # query_en = "a person wearing a helmet riding a motorbike on a street"

    # Chỉ nửa đầu video:
    hits = search_frames(
        video_path, query_vi,
        every_ms=500, topk=10, time_window="first_half"
    )
    for h in hits:
        print(f"Rank {h['rank']} | time {h['time_sec']:.2f}s | score {h['similarity']:.3f}")
        img = Image.open(h["thumb"])
        display(img) 

In [ ]:
!pip install open-clip-torch

In [ ]:
!pip install faiss-cpu
!pip install faiss-gpu